## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [5]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib qt

import cv2
from lib  import CameraCalibrate, EstimateWrapParameterWrapper, IMREAD, warp_image, BinarizeImage, fit_polynomial, search_around_poly, cal_undistort, measure_curvature_real
## Initialize
# initialize paths
calDataPath = 'camera_cal'
testDataPath = 'test_images'

# initialize parameters
mtx, dist  = CameraCalibrate(calDataPath)
M, Minv = EstimateWrapParameterWrapper(mtx, dist, False, True, 'straight_lines1*.jpg')
#M = EstimateWrapParameterWrapper(mtx, dist, True, False)
#M = EstimateWrapParameterWrapper(mtx, dist)



#print(M)
#print(Minv)



FALSE
[[-6.15384615e-01 -1.37820513e+00  9.69230769e+02]
 [ 1.97716240e-16 -1.96794872e+00  8.90769231e+02]
 [ 0.00000000e+00 -2.40384615e-03  1.00000000e+00]]
[[ 1.43118893e-01 -7.85830619e-01  5.61278502e+02]
 [-2.27373675e-16 -5.08143322e-01  4.52638436e+02]
 [-2.41886889e-19 -1.22149837e-03  1.00000000e+00]]


## And so on and so forth...

In [6]:
left_fit = []
right_fit = []
left_fitx_old = []
right_fitx_old = []
ind= 0
cr=[]
pt=[]



def init():
    global left_fit
    global right_fit
    global left_fitx_old
    global right_fitx_old
    global ind
    global cr
    global pt
    left_fit = []
    right_fit = []
    left_fitx_old = []
    right_fitx_old = []
    ind= 0
    cr=[]
    pt=[]
  


def Recast(warped,undist,left_fitx, right_fitx, ploty):
    #global cr
    #global pt
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    ## DEBUG
    #print('pts::')
    #print( color_warp.shape )
    #print(pts)
    #cr =color_warp
    #pt = pts
    cv2.fillPoly(color_warp, np.int_(pts), (0,255, 0))
    newwarp = cv2.warpPerspective(color_warp, Minv, (color_warp.shape[1], color_warp.shape[0])) 
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result


def updateFit(old,new,x_old,x,thres= 50):

    delta = np.mean(np.absolute(x_old-x))
    
    if delta > thres:
        res = old
        ret_X = x_old
    else:
        res = new
        ret_X = x
        
    return res, ret_X

    

def process_image(frame, viz=False, name='None'):
    global left_fit
    global right_fit
    global left_fitx_old
    global right_fitx_old
    global ind
    print('In process_image2')
    
    #plt.imshow(frame)
    
    
    img = cal_undistort(frame, mtx,dist);

    if viz:
        cv2.imwrite('rawdata/'+name +'or.tif',  cv2.cvtColor(frame,cv2.COLOR_BGR2RGB ))
        cv2.imwrite('rawdata/'+name +'or_cal.tif',  cv2.cvtColor(img,cv2.COLOR_BGR2RGB ))
    
    
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB )
    #figure(ind)
    #plt.imshow(img)
    img =  warp_image(img, M)
    

    
    #cv2.imwrite('rawdata/' + str(ind)+'im.tif',  img)
        
    binaryImage = BinarizeImage(img,s_thresh=(170, 255), sx_thresh=(20, 150))
    
   
    
    
    
    #cv2.imwrite('rawdata/' + str(ind)+'msk.tif',  mask*255)
    wrappedBinaryI = binaryImage[:,:,0]+ binaryImage[:,:,1]+binaryImage[:,:,2]
    wrappedBinaryI[wrappedBinaryI>0]=1
    #BI = BI*mask
    #wrappedBinaryI = BI
    
    if viz:
        cv2.imwrite('rawdata/' + name +'or_cal_wr.tif',  cv2.cvtColor(img,cv2.COLOR_BGR2RGB ))
        cv2.imwrite('rawdata/' + name +'or_cal_wr_bin.tif',  wrappedBinaryI*255)


    if len(left_fit) == 0:
        outIM, left_fit, right_fit = fit_polynomial(wrappedBinaryI)
        result, left_fitx, right_fitx, ploty, left_fit_new, right_fit_new, l_fit_2, r_fit_2  = search_around_poly(wrappedBinaryI, left_fit, right_fit)
        left_fitx_old = left_fitx
        right_fitx_old = right_fitx
        
    
    else:
        result, left_fitx, right_fitx, ploty, left_fit_new, right_fit_new, l_fit_2, r_fit_2   = search_around_poly(wrappedBinaryI, left_fit, right_fit)
    
    left_fit, left_fitx_old = updateFit(left_fit,left_fit_new,left_fitx_old, left_fitx, thres= 50)
    right_fit,right_fitx_old = updateFit(right_fit,right_fit_new,right_fitx_old,right_fitx, thres= 50)
        

   
    
    resFinal = Recast(img[:,:,0],frame,left_fitx_old, right_fitx_old, ploty)
    lrad, rrad, posdelta = measure_curvature_real(l_fit_2, r_fit_2, ploty, Minv)
    rad_str = 'Radius of Curvature = '+ str((lrad+rrad)//2  ) + '(m)' 
    if posdelta<0:
        pos = 'left'
    else:
        pos = 'right'
    
    pos_str = 'Vehicle is ' + str(np.absolute( ((posdelta*100)//1)/100 )    ) + 'm ' +pos + ' of center'
    
    resFinal = cv2.putText(resFinal, rad_str, (100,100), cv2.FONT_HERSHEY_SIMPLEX ,  
                   1, (255, 255, 255) , 2, cv2.LINE_AA) 
    resFinal = cv2.putText(resFinal, pos_str, (100,200), cv2.FONT_HERSHEY_SIMPLEX ,  
                   1, (255, 255, 255) , 2, cv2.LINE_AA) 
    
    
       
    if viz:
        cv2.imwrite('rawdata/' + name + 'or_res.tif',  cv2.cvtColor(resFinal,cv2.COLOR_BGR2RGB ) )
    ind+=1
    return resFinal

In [7]:
from moviepy.editor import VideoFileClip
#white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
#temp = 'test_videos_output/solidWhiteRight.mp4'

left_fit = []
right_fit = []

#clip1 = VideoFileClip('project_video.mp4')
clip1 = VideoFileClip('project_video.mp4')

#challenge_video


white_output = 'project_video_out.mp4'

white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False) 

t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

In process_image2
Moviepy - Building video project_video_out.mp4.
Moviepy - Writing video project_video_out.mp4

In process_image2
In process_image2


t:   0%|          | 5/1260 [00:00<01:22, 15.29it/s, now=None]

In process_image2
In process_image2
In process_image2


t:   1%|          | 7/1260 [00:00<01:27, 14.28it/s, now=None]

...

t: 100%|█████████▉| 1256/1260 [01:41<00:00, 11.99it/s, now=None]

In process_image2
In process_image2
In process_image2


t: 100%|██████████| 1260/1260 [01:41<00:00, 12.14it/s, now=None]

In process_image2
In process_image2
In process_image2


Moviepy - Done !
Moviepy - video ready project_video_out.mp4
CPU times: user 3min 9s, sys: 2min 5s, total: 5min 14s
Wall time: 1min 41s


In [4]:
## PIPELINE STARTS HEREwarp_image
fname = testDataPath + '/' + 'test1.jpg';
#fname = calDataPath +  '/' + 'calibration1.jpg';

frame = cv2.imread(fname)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB )
init()
res=process_image(frame, True, 'test1')
plt.imshow(res)

#for i in range(3) :
#    plt.figure(i+1)
#    plt.imshow( frame[:,:,i])



In process_image2


In [14]:
print(  np.linalg.inv(M) )

[[-1.62500000e+00  8.92247511e+00 -6.37286629e+03]
 [ 1.85369521e-15  5.76955903e+00 -5.13934566e+03]
 [ 4.45599811e-18  1.38691323e-02 -1.13541963e+01]]


In [64]:
 a=np.array( [( 91 ,233),
 (419 ,227),
 (410, 324),
 ( 94, 349)], 'int32')
    
#print (a.checkVector(2, CV_32S) )

cv2.fillPoly(cr, a, (0,255, 0))
    

error: OpenCV(4.3.0) /io/opencv/modules/imgproc/src/drawing.cpp:2374: error: (-215:Assertion failed) p.checkVector(2, CV_32S) >= 0 in function 'fillPoly'


In [65]:
a=np.int_(pt)


In [7]:



print(str( ((0.111 * 100)//1)/100 ))


0.11


In [56]:
print(cr.shape)

(720, 1280, 3)


In [5]:
print() )

[-730.76923077 -526.15384615   -0.73076923]


In [10]:
res = np.matmul(M, [550, 480 ,1])
print(res/res[2])

[200. 350.   1.]


In [12]:
print (range(10))


range(0, 10)
